In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

import glob,os
import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf
from tqdm.auto import tqdm
import math
import time
import shutil
import os

import warnings
warnings.filterwarnings("ignore")

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/home/friday_code/miniconda3/envs/champs-infer/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-19 03:23:52.461164: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-19 03:23:52.463757: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-19 03:23:52.511091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the 

In [2]:
class DeepChemTqdmCallback:
    """
    DeepChem-style callback: called as callback(model, current_step).
    Shows a per-epoch tqdm bar (updates once per batch).
    """
    def __init__(self, dataset, batch_size, leave=False):
        self.dataset = dataset
        self.batch_size = int(batch_size)
        self.leave = leave
        # Try to infer dataset length
        try:
            self.n = len(dataset)
        except Exception:
            y = getattr(dataset, "y", None)
            if hasattr(y, "shape"):
                self.n = int(y.shape[0])
            else:
                self.n = None
        self.steps = None if self.n is None else math.ceil(self.n / self.batch_size)
        self.pbar = None
        self.last_epoch = -1

    def __call__(self, model, current_step):
        """
        Called by DeepChem as callback(model, current_step) after each batch.
        current_step is an integer (global batch count).
        """
        # ensure int
        step = int(current_step)

        # If we can't infer steps_per_epoch, show an indeterminate progress spinner
        if self.steps is None:
            if self.pbar is None:
                self.pbar = tqdm(total=None, desc=f"Step {step}", leave=self.leave)
            else:
                self.pbar.update(1)
            return

        # Determine epoch and batch-within-epoch
        epoch = step // self.steps
        batch_in_epoch = step % self.steps

        # If new epoch, close previous bar and open a new one
        if epoch != self.last_epoch:
            if self.pbar is not None:
                try:
                    self.pbar.close()
                except Exception:
                    pass
            self.pbar = tqdm(total=self.steps, desc=f"Epoch {epoch+1}", leave=self.leave)
            self.last_epoch = epoch
            # Update bar to current batch (handles possible non-1 step jumps)
            # Usually first call in epoch will have batch_in_epoch == 0 -> update by 1
            self.pbar.update(batch_in_epoch + 1)
            return

        # Same epoch: advance by 1 (typical case)
        if self.pbar is not None:
            self.pbar.update(1)

    def close(self):
        """Call after training to ensure bar closed."""
        if self.pbar is not None:
            try:
                self.pbar.close()
            except Exception:
                pass
            self.pbar = None

In [3]:
def pred_on_DA_GNN(label, model_dir, val_file):
    Restore_MODEL_DIR = model_dir

    # Featurizerization
    print("# Featurizerization -> ", end="")
    featurizer = dc.feat.ConvMolFeaturizer()
    loader = dc.data.CSVLoader(tasks=[], feature_field="SMILES", featurizer=featurizer)
    testset = loader.create_dataset(val_file, shard_size=10000)
    
    # Model
    val_pred = []
    print("Predicting -> ", end="")
    for i in range(5):
        MODEL_DIR = Restore_MODEL_DIR + '/' + 'loop' + str(i + 1)
        model = dc.models.GraphConvModel(1, mode="regression", model_dir=MODEL_DIR)
        model.restore()
    
        # Predict
        val_pred.append(model.predict(testset))
    
    print("Done")
    val_pred = sum(val_pred) / len(val_pred)
    
    return val_pred

In [ ]:
smiles = [
    "*Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)(F)F)C(F)(F)F)cc3)cc2)cc1",
    "*Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c5ccc(*)cc5)c4)cc3)cc2)cc1",
    "*c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6c5C(=O)N(*)C6=O)cc4C3=O)c2)c1"
]
smiles_df = pd.DataFrame(smiles, columns=['SMILES'])
smiles_df.to_csv('test.csv', index=False)
predictions = {}

for i in ['Tc']:
    print(f"\nPredicting target {i} ...")
    model_dir = f'./trained_models/{i}_GNN_Model/2 layers/'
    predictions[i] = pred_on_DA_GNN(i, model_dir, f"test.csv")
    print(f" -> Completed {i}")


Predicting target FFV ...
# Featurizerization -> Predicting -> WARNING:tensorflow:Detecting that an object or model or tf.train.Checkpoint is being deleted with unrestored values. See the following logs for the specific values in question. To silence these warnings, use `status.expect_partial()`. See https://www.tensorflow.org/api_docs/python/tf/train/Checkpoint#restorefor details about the status object returned by the restore function.
Done
 -> Completed FFV

Predicting target Tc ...
# Featurizerization -> Predicting -> WARNING:tensorflow:6 out of the last 6 calls to <function KerasModel._compute_model at 0x7097585b6c10> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that c

In [6]:
predictions

{'FFV': array([[0.3769389 ],
        [0.37698036],
        [0.3489532 ]], dtype=float32),
 'Tc': array([[0.24284616],
        [0.2750038 ],
        [0.2395732 ]], dtype=float32)}